In [1]:
import scipy as sp
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib
import squarify
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, TfidfTransformer
import re
import xgboost
import scipy
from sklearn.model_selection import train_test_split, GridSearchCV
import sklearn
from sklearn.metrics import mean_squared_log_error
from sklearn.linear_model import SGDRegressor
from sklearn.pipeline import Pipeline

In [2]:
data_train = pd.read_csv("train.tsv", sep='\t', header=0)
data_train.head()

,train_id,name,item_condition_id,category_name,brand_name,price,shipping,item_description
0,0,MLB Cincinnati Reds T Shirt Size XL,3,Men/Tops/T-shirts,NaN,10.0,1,No description yet
1,1,Razer BlackWidow Chroma Keyboard,3,Electronics/Computers & Tablets/Components & P...,Razer,52.0,0,This keyboard is in great condition and works ...
2,2,AVA-VIV Blouse,1,Women/Tops & Blouses/Blouse,Target,10.0,1,Adorable top with a hint of lace and a key hol...
3,3,Leather Horse Statues,1,Home/Home Décor/Home Décor Accents,NaN,35.0,1,New with tags. Leather horses. Retail for [rm]...
4,4,24K GOLD plated rose,1,Women/Jewelry/Necklaces,NaN,44.0,0,Complete with certificate of authenticity


In [3]:
data_test = pd.read_csv("test.tsv", sep='\t', header=0)
data_test.head()

,test_id,name,item_condition_id,category_name,brand_name,shipping,item_description
0,0,"Breast cancer ""I fight like a girl"" ring",1,Women/Jewelry/Rings,NaN,1,Size 7
1,1,"25 pcs NEW 7.5""x12"" Kraft Bubble Mailers",1,Other/Office supplies/Shipping Supplies,NaN,1,"25 pcs NEW 7.5""x12"" Kraft Bubble Mailers Lined..."
2,2,Coach bag,1,Vintage & Collectibles/Bags and Purses/Handbag,Coach,1,Brand new coach bag. Bought for [rm] at a Coac...
3,3,Floral Kimono,2,Women/Sweaters/Cardigan,NaN,0,-floral kimono -never worn -lightweight and pe...
4,4,Life after Death,3,Other/Books/Religion & Spirituality,NaN,1,Rediscovering life after the loss of a loved o...


In [4]:
data = pd.concat([data_train, data_test], 0)
nrow_train = data_train.shape[0]
print(nrow_train)
y  = np.log1p(data_train["price"])
y_train = np.log1p(data_train["price"])

1482535


In [5]:
data['primary_cat'] = data.category_name.str.extract('([^/]+)/[^/]+/[^/]+')
data['secondary_cat'] = data.category_name.str.extract('[^/]+/([^/]+/[^/]+)')

c:\users\kj.park\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  """Entry point for launching an IPython kernel.
c:\users\kj.park\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  


In [6]:
non_text = [',','-','.','&','%','#', '(',')','[',']','*','!','✨','▪️','"','~',"'", '<', '>', '®','™']
for symbols in non_text: 
    data['primary_cat'] = data['primary_cat'].str.replace(symbols, '') 
    data['secondary_cat'] = data['secondary_cat'].str.replace(symbols, '') 
    data['item_description'] = data['item_description'].str.replace(symbols, '') 
    data['brand_name'] = data['brand_name'].str.replace(symbols, '')
    data['name'] = data['name'].str.replace(symbols, '')
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2175894 entries, 0 to 693358
Data columns (total 11 columns):
brand_name           object
category_name        object
item_condition_id    int64
item_description     object
name                 object
price                float64
shipping             int64
test_id              float64
train_id             float64
primary_cat          object
secondary_cat        object
dtypes: float64(3), int64(2), object(6)
memory usage: 199.2+ MB


In [7]:
data = data.apply(lambda x: x.astype(str).str.lower())

In [8]:
data.head()

,brand_name,category_name,item_condition_id,item_description,name,price,shipping,test_id,train_id,primary_cat,secondary_cat
0,nan,men/tops/t-shirts,3,no description yet,mlb cincinnati reds t shirt size xl,10.0,1,nan,0.0,men,tops/tshirts
1,razer,electronics/computers & tablets/components & p...,3,this keyboard is in great condition and works ...,razer blackwidow chroma keyboard,52.0,0,nan,1.0,electronics,computers tablets/components parts
2,target,women/tops & blouses/blouse,1,adorable top with a hint of lace and a key hol...,avaviv blouse,10.0,1,nan,2.0,women,tops blouses/blouse
3,nan,home/home décor/home décor accents,1,new with tags leather horses retail for rm eac...,leather horse statues,35.0,1,nan,3.0,home,home décor/home décor accents
4,nan,women/jewelry/necklaces,1,complete with certificate of authenticity,24k gold plated rose,44.0,0,nan,4.0,women,jewelry/necklaces


In [9]:
features = [data['name'], data['brand_name'] ,data['item_description'] ,data['primary_cat'], data['secondary_cat']]
for x in features:
    x = x.fillna(value = 'none', inplace=False, axis=0)
    print(x.isnull().sum())

0
0
0
0
0


In [10]:
def tokenizer(text):
    if text:
        result = re.findall('[a-z]{2,}', text.lower())
    else:
        result = []
    return result

In [11]:
#combining all text values within one variable X_text
X_text = (data['name'] + ' ' + data['brand_name'] + ' ' +
          data['item_description'] + ' ' + data['primary_cat'] + ' ' + 
          data['secondary_cat']).values

In [12]:
X_text

array([ 'mlb cincinnati reds t shirt size xl nan no description yet men tops/tshirts',
       'razer blackwidow chroma keyboard razer this keyboard is in great condition and works like it came out of the box all of the ports are tested and work perfectly the lights are customizable via the razer synapse app on your pc electronics computers  tablets/components  parts',
       'avaviv blouse target adorable top with a hint of lace and a key hole in the back the pale pink is a 1x and i also have a 3x available in white women tops  blouses/blouse',
       ...,
       'galaxy s8 hard shell case nan new free shipping basstop case electronics cell phones  accessories/cases covers  skins',
       'hi low floral kimono nan floral kimono tropical print open front hi low design great for layering or as a cover up women swimwear/coverups',
       'freeship 2 floral scrub tops medium nan 2 floral scrub tops worn less than 5 times each brown has a belt tie women tops  blouses/tshirts'], dtype=object

In [13]:
#instantiating a TFidVectorizer
count_desc = TfidfVectorizer(tokenizer=tokenizer, max_features = 50000, ngram_range = (1,3),stop_words = "english")

In [14]:
X_text = count_desc.fit_transform(X_text)

In [15]:
X_text

<2175894x50000 sparse matrix of type '<class 'numpy.float64'>'
	with 78711917 stored elements in Compressed Sparse Row format>

In [16]:
#Getting dummies for the non_text values for item_condition with range (1-5) and shipping (0,1)
X_dummies = scipy.sparse.csr_matrix(pd.get_dummies(data[[
    "item_condition_id", "shipping"]], sparse = True).values)

In [17]:
#re-combining transformed features into one X value:
X = scipy.sparse.hstack((X_dummies, X_text)).tocsr()

#IGNORE FOR NOW
count = CountVectorizer()
vect = LabelBinarizer(sparse_output=True)
count_desc = TfidfVectorizer(max_features = 50000, ngram_range = (1,3),stop_words = "english")
X_desc = count_desc.fit_transform(data["item_description"])
X_name = count.fit_transform(data['name'])
X_cat_1 = count.fit_transform(data['primary_cat'])
X_cat_2 = count.fit_transform(data['secondary_cat'])
X_brand = vect.fit_transform(data['brand_name'])
X_dummies = scipy.sparse.csr_matrix(pd.get_dummies(data[[
    "item_condition_id", "shipping"]], sparse = True).values)

#IGNORE FOR NOW
X = scipy.sparse.hstack((X_dummies, 
                         X_desc,
                         X_brand,
                         X_cat_1,
                         X_cat_2,
                         X_name)).tocsr()

In [18]:
X_training = X[:nrow_train]
X_testing = X[nrow_train:]

In [19]:
print(X_training.shape)
print(y.shape)

(1482535, 50007)
(1482535,)


In [20]:
X_train, X_test, y_train, y_test = train_test_split(X_training, y, test_size=0.4, random_state=42)

In [21]:
print(X_train.shape)
print(y_train.shape)

(889521, 50007)
(889521,)


In [22]:
model = SGDRegressor(loss='squared_loss', penalty='l2', random_state=101, max_iter=5)
params = {'penalty':['none','l2','l1'],
          'alpha':[1e-4, 2e-4, 5e-4, 1e-3, 2e-3, 5e-3, 1e-2, 2e-2, 5e-2, 0.1]}
gscv = GridSearchCV(estimator=model,
                  param_grid=params,
                  scoring='neg_mean_squared_log_error',
                  n_jobs=1,
                  cv=5,
                  verbose=3)

gscv.fit(X_train, y_train)

Fitting 5 folds for each of 30 candidates, totalling 150 fits
[CV] alpha=0.0001, penalty=none ......................................
[CV]  alpha=0.0001, penalty=none, score=-0.021727306009160403, total=   2.9s
[CV] alpha=0.0001, penalty=none ......................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    3.2s remaining:    0.0s


[CV]  alpha=0.0001, penalty=none, score=-0.021748110995804055, total=   3.0s
[CV] alpha=0.0001, penalty=none ......................................


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    6.5s remaining:    0.0s


[CV]  alpha=0.0001, penalty=none, score=-0.021672166545402768, total=   2.8s
[CV] alpha=0.0001, penalty=none ......................................
[CV]  alpha=0.0001, penalty=none, score=-0.021695815271642053, total=   3.0s
[CV] alpha=0.0001, penalty=none ......................................
[CV]  alpha=0.0001, penalty=none, score=-0.021466765028354304, total=   2.8s
[CV] alpha=0.0001, penalty=l2 ........................................
[CV]  alpha=0.0001, penalty=l2, score=-0.021947404003389875, total=   2.6s
[CV] alpha=0.0001, penalty=l2 ........................................
[CV]  alpha=0.0001, penalty=l2, score=-0.021960787006276395, total=   2.6s
[CV] alpha=0.0001, penalty=l2 ........................................
[CV]  alpha=0.0001, penalty=l2, score=-0.021885447894606426, total=   2.6s
[CV] alpha=0.0001, penalty=l2 ........................................
[CV]  alpha=0.0001, penalty=l2, score=-0.021909051725762872, total=   4.8s
[CV] alpha=0.0001, penalty=l2 .............

[CV]  alpha=0.001, penalty=l1, score=-0.02948696966628886, total=   8.8s
[CV] alpha=0.002, penalty=none .......................................
[CV]  alpha=0.002, penalty=none, score=-0.021727306009160403, total=   6.5s
[CV] alpha=0.002, penalty=none .......................................
[CV]  alpha=0.002, penalty=none, score=-0.021748110995804055, total=   5.8s
[CV] alpha=0.002, penalty=none .......................................
[CV]  alpha=0.002, penalty=none, score=-0.021672166545402768, total=   6.1s
[CV] alpha=0.002, penalty=none .......................................
[CV]  alpha=0.002, penalty=none, score=-0.021695815271642053, total=   6.2s
[CV] alpha=0.002, penalty=none .......................................
[CV]  alpha=0.002, penalty=none, score=-0.021466765028354304, total=   5.8s
[CV] alpha=0.002, penalty=l2 .........................................
[CV]  alpha=0.002, penalty=l2, score=-0.025424905540928112, total=   6.5s
[CV] alpha=0.002, penalty=l2 ..................

[CV]  alpha=0.02, penalty=l1, score=-0.03282177518565353, total=   7.9s
[CV] alpha=0.02, penalty=l1 ..........................................
[CV]  alpha=0.02, penalty=l1, score=-0.032818422060498335, total=   7.9s
[CV] alpha=0.02, penalty=l1 ..........................................
[CV]  alpha=0.02, penalty=l1, score=-0.032766597840518163, total=   9.2s
[CV] alpha=0.02, penalty=l1 ..........................................
[CV]  alpha=0.02, penalty=l1, score=-0.032630885057433376, total=   8.3s
[CV] alpha=0.05, penalty=none ........................................
[CV]  alpha=0.05, penalty=none, score=-0.021727306009160403, total=   5.5s
[CV] alpha=0.05, penalty=none ........................................
[CV]  alpha=0.05, penalty=none, score=-0.021748110995804055, total=   5.8s
[CV] alpha=0.05, penalty=none ........................................
[CV]  alpha=0.05, penalty=none, score=-0.021672166545402768, total=   5.8s
[CV] alpha=0.05, penalty=none ............................

[Parallel(n_jobs=1)]: Done 150 out of 150 | elapsed: 22.6min finished


GridSearchCV(cv=5, error_score='raise',
       estimator=SGDRegressor(alpha=0.0001, average=False, epsilon=0.1, eta0=0.01,
       fit_intercept=True, l1_ratio=0.15, learning_rate='invscaling',
       loss='squared_loss', max_iter=5, n_iter=None, penalty='l2',
       power_t=0.25, random_state=101, shuffle=True, tol=None, verbose=0,
       warm_start=False),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'penalty': ['none', 'l2', 'l1'], 'alpha': [0.0001, 0.0002, 0.0005, 0.001, 0.002, 0.005, 0.01, 0.02, 0.05, 0.1]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='neg_mean_squared_log_error', verbose=3)

In [23]:
predictions = gscv.predict(X_test)

In [24]:
predictions

array([ 2.4755281 ,  2.93839098,  2.31612233, ...,  2.58367158,
        2.8324403 ,  4.2656816 ])

In [25]:
from sklearn.metrics import classification_report
report = classification_report(y_test, predictions)
print(report)

ValueError: Unknown label type: (777341     2.564949
1463629    4.343805
350669     2.564949
310222     1.386294
759257     3.433987
288846     5.420535
1178450    2.833213
726296     2.564949
840510     2.772589
1473033    2.890372
1170796    3.367296
506473     2.302585
1134656    2.995732
587885     1.609438
93928      2.708050
260369     4.276666
96202      2.397895
486334     2.708050
1437987    4.394449
1343222    2.890372
1403115    3.044522
1393495    2.564949
1364847    3.367296
1339539    2.564949
1262505    3.178054
811065     3.610918
310616     3.828641
828784     3.091042
1385423    3.218876
850832     1.945910
             ...   
959437     2.708050
777685     5.525453
477321     2.484907
1337172    2.079442
1040033    2.564949
642740     3.258097
991497     3.295837
5432       3.737670
396647     2.772589
1313722    3.218876
626140     3.401197
305116     2.944439
164473     2.708050
847060     2.944439
1014054    2.564949
1083534    3.784190
1182515    2.302585
1230732

In [ ]:
test["price"] = np.expm1(predictions)
test['test_id'] = 
test[["test_id", "price"]].to_csv("kyungjunism_submission_xgb.csv", index = False)